In [ ]:
import copy

EMPTY = '.'
WHITE = 'W'
BLACK = 'B'

class Checkers:
    def __init__(self):
        self.board = self.create_board()
        self.current_player = WHITE

    def create_board(self):
        board = [[EMPTY for _ in range(8)] for _ in range(8)]
        for row in range(3):
            for col in range(8):
                if (row + col) % 2 == 1:
                    board[row][col] = BLACK
        for row in range(5, 8):
            for col in range(8):
                if (row + col) % 2 == 1:
                    board[row][col] = WHITE
        return board

    def print_board(self):
        print("  " + " ".join(str(i) for i in range(8)))
        for idx, row in enumerate(self.board):
            print(f"{idx} " + " ".join(row))
        print()

    def get_all_moves(self, player):
        moves = []
        for r in range(8):
            for c in range(8):
                if self.board[r][c] == player:
                    moves.extend(self.get_piece_moves(r, c, player))
        return moves

    def get_piece_moves(self, r, c, player):
        moves = []
        directions = [(-1, -1), (-1, 1)] if player == WHITE else [(1, -1), (1, 1)]
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if 0 <= nr < 8 and 0 <= nc < 8 and self.board[nr][nc] == EMPTY:
                moves.append(((r, c), (nr, nc)))
            # Capture
            nr2, nc2 = r + 2*dr, c + 2*dc
            if (0 <= nr2 < 8 and 0 <= nc2 < 8 and
                self.board[nr][nc] != EMPTY and self.board[nr][nc] != player and
                self.board[nr2][nc2] == EMPTY):
                moves.append(((r, c), (nr2, nc2)))
        return moves

    def make_move(self, move):
        (r1, c1), (r2, c2) = move
        player = self.board[r1][c1]
        self.board[r1][c1] = EMPTY
        self.board[r2][c2] = player
        # Check for capture
        if abs(r2 - r1) == 2:
            self.board[(r1 + r2) // 2][(c1 + c2) // 2] = EMPTY

    def evaluate(self):
        white_pieces = sum(row.count(WHITE) for row in self.board)
        black_pieces = sum(row.count(BLACK) for row in self.board)
        return black_pieces - white_pieces

    def is_game_over(self):
        return not self.get_all_moves(WHITE) or not self.get_all_moves(BLACK)

def minimax(board, depth, alpha, beta, maximizing_player):
    if depth == 0 or board.is_game_over():
        return board.evaluate(), None

    if maximizing_player:
        max_eval = float('-inf')
        best_move = None
        for move in board.get_all_moves(BLACK):
            new_board = copy.deepcopy(board)
            new_board.make_move(move)
            eval, _ = minimax(new_board, depth - 1, alpha, beta, False)
            if eval > max_eval:
                max_eval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = float('inf')
        best_move = None
        for move in board.get_all_moves(WHITE):
            new_board = copy.deepcopy(board)
            new_board.make_move(move)
            eval, _ = minimax(new_board, depth - 1, alpha, beta, True)
            if eval < min_eval:
                min_eval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move

def play_game():
    game = Checkers()
    while not game.is_game_over():
        game.print_board()
        if game.current_player == WHITE:
            moves = game.get_all_moves(WHITE)
            if not moves:
                print("No moves available for White. Black wins!")
                break
            print("Available moves:")
            for idx, move in enumerate(moves):
                print(f"{idx}: {move}")
            idx = int(input("Select move index: "))
            move = moves[idx]
            game.make_move(move)
            game.current_player = BLACK
        else:
            print("AI is thinking...")
            _, move = minimax(game, 4, float('-inf'), float('inf'), True)
            if move is None:
                print("No moves available for Black. White wins!")
                break
            print(f"AI selects move: {move}")
            game.make_move(move)
            game.current_player = WHITE
    game.print_board()
    print("Game over.")

if __name__ == "__main__":
    play_game()


In [ ]:
def minimax(cards, left, right, is_max_turn, alpha, beta):
    if left > right:
        return 0
    if is_max_turn:
        pick_left = cards[left] + minimax(cards, left + 1, right, False, alpha, beta)
        pick_right = cards[right] + minimax(cards, left, right - 1, False, alpha, beta)
        return max(pick_left, pick_right)
    else:
        pick_left = minimax(cards, left + 1, right, True, alpha, beta)
        pick_right = minimax(cards, left, right - 1, True, alpha, beta)
        return min(pick_left, pick_right)

def play_card_game():
    cards = [4, 10, 6, 2, 9, 5]
    max_score = 0
    min_score = 0
    left = 0
    right = len(cards) - 1
    is_max_turn = True

    while left <= right:
        if is_max_turn:
            pick_left = cards[left] + minimax(cards, left + 1, right, False, float('-inf'), float('inf'))
            pick_right = cards[right] + minimax(cards, left, right - 1, False, float('-inf'), float('inf'))
            if pick_left >= pick_right:
                print(f"Max picks {cards[left]}")
                max_score += cards[left]
                left += 1
            else:
                print(f"Max picks {cards[right]}")
                max_score += cards[right]
                right -= 1
        else:
            if cards[left] <= cards[right]:
                print(f"Min picks {cards[left]}")
                min_score += cards[left]
                left += 1
            else:
                print(f"Min picks {cards[right]}")
                min_score += cards[right]
                right -= 1
        is_max_turn = not is_max_turn

    print(f"Final Scores - Max: {max_score}, Min: {min_score}")
    if max_score > min_score:
        print("Winner: Max")
    elif min_score > max_score:
        print("Winner: Min")
    else:
        print("It's a tie!")

if __name__ == "__main__":
    play_card_game()



In [ ]:
import random

GRID_SIZE = 10
SHIP_SIZES = [5, 4, 3, 3, 2]

def create_grid():
    return [['~' for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

def print_grid(grid):
    print("  " + " ".join(str(i) for i in range(GRID_SIZE)))
    for idx, row in enumerate(grid):
        print(f"{idx} " + " ".join(row))
    print()

def place_ships_randomly(grid):
    for size in SHIP_SIZES:
        placed = False
        while not placed:
            orientation = random.choice(['H', 'V'])
            if orientation == 'H':
                row = random.randint(0, GRID_SIZE - 1)
                col = random.randint(0, GRID_SIZE - size)
                if all(grid[row][col + i] == '~' for i in range(size)):
                    for i in range(size):
                        grid[row][col + i] = 'S'
                    placed = True
            else:
                row = random.randint(0, GRID_SIZE - size)
                col = random.randint(0, GRID_SIZE - 1)
                if all(grid[row + i][col] == '~' for i in range(size)):
                    for i in range(size):
                        grid[row + i][col] = 'S'
                    placed = True

def get_probability_grid(guess_grid):
    prob_grid = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

::contentReference[oaicite:0]{index=0}

